# Import Required Libraries
Import numpy, matplotlib.pyplot, solve_ivp from scipy.integrate, and interact and FloatSlider from ipywidgets.

In [231]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider, Button, HBox, VBox, interactive_output
import mplcursors
from functools import partial
from matplotlib.lines import Line2D
from scipy.optimize import root_scalar
import sympy as sp

# plots in dark mode
#plt.style.use('dark_background')

# Define the Conformity Function
Define the function f(x, alpha) that models the conformity behavior.

In [232]:
# Define the Conformity Function
def f(x, alpha):
    if x <= 0.5:
        return (((2 * x) ** (alpha)) / 2)
    else:
        return (1 - ((2 * (1 - x)) ** (alpha)) / 2)

# Vectorize f(x) for plotting
f_vec = np.vectorize(f)

# Define the Dynamical System
Define the function dynamics(t, y, s, m, alpha) that models the dynamical system.

In [233]:
# Define the Dynamical System
def dynamics(t, y, s, m, alpha):
    x_i, x_s = y
    x = (1 - s) * x_i + s * x_s
    dx_i_dt = m - x_i
    dx_s_dt = f(x, alpha) - x_s
    return [dx_i_dt, dx_s_dt]

# Find the Fixed Point(s)
Set $x_i=m$ and solve for $f((1-s)m+sx_s)=x_s$ given values of $\alpha,s,m$

In [234]:
def find_all_roots(s, m, alpha, num_intervals=100):
    def root_func(x_s, s, m, alpha):
        return f((1 - s) * m + s * x_s, alpha) - x_s
    roots = []
    x_vals = np.linspace(0, 1, num_intervals + 1)
    for i in range(num_intervals):
        x0, x1 = x_vals[i], x_vals[i + 1]
        try:
            result = root_scalar(root_func, args=(s, m, alpha), bracket=[x0, x1], method='brentq')
            if result.converged:
                root = result.root
                # Check if the root is already in the list (to avoid duplicates)
                if not any(np.isclose(root, r) for r in roots):
                    roots.append(root)
        except ValueError:
            # No root found in this subinterval
            pass
    return roots

    

## Compute the Jacobian at the fixed points

The Jacobian is 
$$J(x_i,x_s)=\begin{pmatrix}-1&0\\f'(x)(1-s)&f'(x)s-1\end{pmatrix}$$
where $x=(1-s)x_i+sx_s$.


In [235]:
def f_prime(x, alpha):
    if x <= 0.5:
        return alpha * (2 * x)**(alpha - 1)
    else:
        return alpha * (2 * (1 - x))**(alpha - 1)

def get_jacobians(s, m, alpha):
    roots = find_all_roots(s, m, alpha)
    jacobians = []
    for root in roots:
        x_val = (1 - s) * m + s * root
        f_prime_val = f_prime(x_val, alpha)
        jacobian = np.array([[-1, 0], [f_prime_val * (1-s), f_prime_val * s - 1]])
        jacobians.append(jacobian)
    return jacobians

def analyze_jacobian(jacobian):
    trace = np.trace(jacobian)
    # For this matrix, -1 is always an eigenvalue. The other eigenvalue is the trace minus -1
    ew = trace + 1
    # get element on second row and first column
    a = jacobian[1, 0]
    if ew == -1 and a != 0:
        return 'Defective stable node'
    elif ew == -1 and a == 0:
        return 'Stable star'
    elif ew != -1 and ew < 0:
        return 'Stable node'
    elif ew == 0:
        return 'Unknown'
    elif ew > 0:
        return 'Saddle'
    
def analyze_equilibrium(s, m, alpha):
    roots = find_all_roots(s, m, alpha)
    jacobians = get_jacobians(s, m, alpha)
    # we want to display each equilibrium point, the type of equilibrium point, and the jacobian matrix
    results = []
    for i, root in enumerate(roots):
        jacobian = jacobians[i]
        result = {'root': root, 'type': analyze_jacobian(jacobian), 'jacobian': jacobian}
        results.append(result)
    return results

def display_equilibrium_points(s, m, alpha):
    results = analyze_equilibrium(s, m, alpha)
    # make sure all numerical values are displayed with 2 decimal places
    np.set_printoptions(precision=2)
    for result in results:
        j22 = result['jacobian'][1, 1]
        print(f'Equilibrium point: {result["root"]:.4f}')
        print(f'Type: {result["type"]}')
        print(f'Jacobian matrix (x_s={result["root"]:.2f}): eigenvalues [-1,{j22:.3f}]\n{result["jacobian"]}')
        print()

# Plot Function with Interactive Sliders
Define the function plot_system $(s, m, \alpha, x_{i0}, x_{s0})$ and use interact to create interactive sliders for the parameters.

In [236]:
# Plot Function with Interactive Sliders
def plot_f_and_x(s, m, alpha, t, x_t):
    # we create a subplot with 2 columns and 1 row
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))
    # we plot f(x) in the first subplot
    x_vals = np.linspace(0, 1, 300)
    f_vals = f_vec(x_vals, alpha)
    axs[0].plot(x_vals, f_vals, 'c-', label='f(x)')
    # plot (m,f(m)) as a red point
    axs[0].scatter([m], [f(m, alpha)], color='red', label='Optimal Point (m={}, f(m)={:.2f})'.format(m, f(m, alpha)))
    # label the point with text
    #axs[0].text(m, f(m, alpha), f'(m, f(m)) = ({m:.2f}, {f(m, alpha):.2f})', fontsize=12, ha='right', va='bottom')
    axs[0].set_xlabel('x')
    axs[0].set_ylabel('f(x)')
    axs[0].set_title('Conformity Function f(x) for alpha={:.2f}'.format(alpha))
    axs[0].grid()
    axs[0].legend()
    # we plot x(t) in the second subplot
    # Plot x(t) = (1-s)x_i(t) + s x_s(t) over time
    axs[1].plot(t, x_t, label='x(t) = (1-s)x_i(t) + s x_s(t)', color='orange')
    # plot the horizontal lines at the fixed points
    roots = find_all_roots(s, m, alpha)
    for root in roots:
        # the fixed point for x will be (1-s)m + s*root
        x_val = (1-s)*m + s*root
        if len(roots) == 3:
            # we want to label the fixed points as stable or unstable
            # the middle fixed point is always unstable and the other two are stable
            if root == roots[1]:
                axs[1].axhline(y=x_val, color='black', linestyle='--', label=f'x={x_val:.3f} (unstable)')
            else:
                axs[1].axhline(y=x_val, color='black', linestyle='--', label=f'x={x_val:.3f} (stable)')
        else:
            axs[1].axhline(y=x_val, color='black', linestyle='--', label=f'x={x_val:.3f}')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('x(t)')
    axs[1].set_title('Trajectory of x(t) over time')
    axs[1].axhline(y=0.5, color='r', linestyle='--', label='x=0.5')
    axs[1].axhline(y=m, color='g', linestyle='--', label='x=m={:.2f}'.format(m))
    axs[1].set_ylim(0, 1)
    axs[1].grid()
    axs[1].legend()
    
    # Connect click event
    plt.show()
    
def plot_system(s, m, alpha, x_i0, x_s0, save=False):
    t_final = 999
    t_span = (0, t_final)
    y0 = [x_i0, x_s0]
    sol = solve_ivp(dynamics, t_span, y0, args=(s, m, alpha), dense_output=True)
    t = np.linspace(t_span[0], t_span[1], 300)
    x_i, x_s = sol.sol(t)
    x_t = (1 - s) * x_i + s * x_s


    plt.figure(figsize=(10, 6))
    plt.plot([m, m], [0, 1], 'r--', label='x_i nullcline (x_i = m)')
    x_s_vals = np.linspace(0, 1, 300)
    x_i_vals = np.linspace(0, 1, 300)
    X_i, X_s = np.meshgrid(x_i_vals, x_s_vals)
    X = (1-s)*X_i + s*X_s
    F_X_vals = f_vec(X, alpha)
    contour = plt.contour(X_i, X_s, F_X_vals - X_s, levels=[0], colors='lightblue', linestyles='dashed')
    
    # print all fixed points and initial and terminal points
    roots = find_all_roots(s, m, alpha)
    # plots red x's at the roots (m,root) for each root
    for root in roots:
        plt.scatter([m], [root], color='red', marker='x', label='Fixed Point (m, {})'.format(round(root, 3)))
    # print roots up to 3 decimal places
    roots = [round(r, 3) for r in roots]
    #print(f'x_s fixed Points (s={s}, m={m}, alpha={alpha:.2f}):\n{roots}')
    display_equilibrium_points(s, m, alpha)
    print(f'x_i(t=0) = {x_i0:.2f}, x_s(t=0) = {x_s0:.2f}, x(t=0) = {(1-s)*x_i0 + s*x_s0:.2f}')
    print(f'x_i(t=∞) = {x_i[-1]:.2f}, x_s(t=∞) = {x_s[-1]:.2f}, x(t=∞) = {(1-s)*x_i[-1] + s*x_s[-1]:.2f}')
    
    # Trajectory and initial point
    plt.plot(x_i, x_s, 'g-', label='Trajectory')
    plt.scatter([x_i0], [x_s0], color='purple', label='Initial Point')
    # put text on the initial point
    x_initial = (1 - s) * x_i0 + s * x_s0
    #plt.text(x_i0, x_s0, f'(x_i(0)={x_i0:.2f}, x_s(0)={x_s0:.2f})\n x(0)={x_initial:.2f}', fontsize=12, ha='right', va='bottom')
    
    # plot terminal point (t=t_final)
    x_terminal = (1 - s) * x_i[-1] + s * x_s[-1]
    #plt.text(x_i[-1], x_s[-1], f'(x_i*={x_i[-1]:.2f}, x_s*={x_s[-1]:.2f})\n x*={x_terminal:.2f}', fontsize=12, ha='right', va='bottom')
    plt.scatter([x_i[-1]], [x_s[-1]], color='blue', label='Terminal Point')
    
    plt.scatter([m], [m], color='orange', label='Optimal Point (m, m)')
    plt.xlabel('x_i (Individual Learners)')
    plt.ylabel('x_s (Social Learners)')
    plt.title(f'Plot (s={s}, m={m}, alpha={alpha:.2f}): x* = {x_terminal:.2f}')

    # Create a proxy artist for the contour plot
    proxy = Line2D([0], [0], linestyle='dashed', color='lightblue')

    # Combine the handles and labels for the legend
    handles, labels = plt.gca().get_legend_handles_labels()
    handles = [proxy] + handles  # Add the proxy artist at the beginning
    labels = ['x_s nullcline (f(x) = x_s)'] + labels  # Add corresponding label at the beginning

    # Create a single legend
    plt.legend(handles=handles, labels=labels, loc='lower right')
    plt.grid()
    
    if save:
        filename = f'img/plot_s_{s}_m_{m}_alpha_{alpha:.2f}_xi0_{x_i0}_xs0_{x_s0}.png'
        plt.savefig(filename)
        print(f"Plot saved as {filename}")
    
    plt.show()
    plot_f_and_x(s, m, alpha, t, x_t)

# Define sliders
s_slider = FloatSlider(value=0.96, min=0, max=1, step=0.01, description='s')
m_slider = FloatSlider(value=0.35, min=0, max=1, step=0.01, description='m')
alpha_slider = FloatSlider(value=1.1, min=0.0, max=2, step=0.01, description='alpha')
x_i0_slider = FloatSlider(value=0.45, min=0, max=1, step=0.01, description='x_i(0)')
x_s0_slider = FloatSlider(value=0.68, min=0, max=1, step=0.01, description='x_s(0)')

# Save button
save_button = Button(description="Save Plot")

# Function to handle saving
def save_plot_callback(_):
    plot_system(s_slider.value, m_slider.value, alpha_slider.value, x_i0_slider.value, x_s0_slider.value, save=True)

# Attach save button callback
save_button.on_click(save_plot_callback)

# Use interactive_output to link sliders to the plotting function
interactive_plot = interactive_output(plot_system, {
    's': s_slider,
    'm': m_slider,
    'alpha': alpha_slider,
    'x_i0': x_i0_slider,
    'x_s0': x_s0_slider
})

# Combine sliders and button
ui = VBox([s_slider, m_slider, alpha_slider, x_i0_slider, x_s0_slider, save_button])

# Display the interactive output with the manually created UI
display(ui, interactive_plot)

Output()